In [1]:
# Clean and prepare a bugged_#.txt file for JSON inclusion

import shutil
import json
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

def clean_and_convert(txt_path: str, json_out_dir: str):
    txt_file = Path(txt_path)
    assert txt_file.exists(), f"File not found: {txt_file}"

    # Create backup
    bak_path = txt_file.with_suffix('.bak')
    shutil.copy(txt_file, bak_path)
    print(f"🛡️  Backup created: {bak_path}")

    # Read and clean
    with open(txt_file, encoding='utf-8') as f:
        raw_lines = f.readlines()

    non_blank_lines = [line.rstrip('\n') for line in raw_lines if line.strip()]
    removed = len(raw_lines) - len(non_blank_lines)

    # Overwrite cleaned version
    with open(txt_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(non_blank_lines) + '\n')

    print(f"🧼 Cleaned {txt_file.name}: removed {removed} blank lines")

    # Prepare JSON snapshot
    snapshot_name = txt_file.stem
    snapshot_json = {
        "snapshot": snapshot_name,
        "lines": non_blank_lines
    }

    out_dir = Path(json_out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{snapshot_name}.json"

    with open(out_path, 'w', encoding='utf-8') as out:
        json.dump(snapshot_json, out, indent=2, ensure_ascii=False)

    print(f"📁 Saved {out_path} ({len(non_blank_lines)} lines)")

# Interactive dropdown for filenames in directory
def interactive_cleaner(txt_dir, json_out_dir):
    path = Path(txt_dir)
    files = sorted([f.name for f in path.glob("bugged_*.txt")])
    dropdown = widgets.Dropdown(options=files, description='File:')
    button = widgets.Button(description="Clean & Convert")

    def on_click(b):
        clean_and_convert(str(path / dropdown.value), json_out_dir)

    button.on_click(on_click)
    display(dropdown, button)

# Example usage in notebook:
# interactive_cleaner(
#     "D:/gpt_log/GPT bugged conversation/Bruno",
#     "D:/shared_project/gpt_log/Bruno/bugged_json"
# )
